In [65]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [66]:
import datetime
import numpy as np

In [70]:
import pandas as pd

train = pd.read_excel("/content/drive/MyDrive/ufo_hack/train.xlsx")
test = pd.read_excel("/content/drive/MyDrive/ufo_hack/test.xlsx")
macro = pd.read_excel("/content/drive/MyDrive/ufo_hack/Макропоказатели.xlsx")
metal = pd.read_excel("/content/drive/MyDrive/ufo_hack/Показатели рынка металла.xlsx")
price = pd.read_excel("/content/drive/MyDrive/ufo_hack/Цены на сырье.xlsx")
mater = pd.read_excel("/content/drive/MyDrive/ufo_hack/Данные рынка стройматериалов.xlsx")
fuel = pd.read_excel("/content/drive/MyDrive/ufo_hack/Топливо.xlsx")
lme = pd.read_excel("/content/drive/MyDrive/ufo_hack/Индекс LME.xlsx")
chmf = pd.read_csv("/content/drive/MyDrive/ufo_hack/CHMF Акции.csv")
magn = pd.read_csv("/content/drive/MyDrive/ufo_hack/MAGN Акции.csv")
nlmk = pd.read_csv("/content/drive/MyDrive/ufo_hack/NLMK Акции.csv")

In [71]:
lme = lme.dropna()
lme.rename(columns={'дата':'dt', 'цена':'Цена LME'}, inplace=True)

<ipython-input-71-e9c8a8d7c16f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lme.rename(columns={'дата':'dt', 'цена':'Цена LME'}, inplace=True)


In [72]:
chmf = chmf[['Date', 'Price', 'Vol.']]
chmf.rename(columns = {'Date':'dt', 'Price':'Цена CHMF', 'Vol.':'Объем CHMF'}, inplace=True )
magn = magn[['Дата', 'Цена', 'Объём']]
magn.rename(columns = {'Дата':'dt', 'Цена':'Цена MAGN', 'Объём':'Объем MAGN'}, inplace=True )
nlmk = nlmk[['Date', 'Price', 'Vol.']]
nlmk.rename(columns = {'Date':'dt', 'Price':'Цена NLMK', 'Vol.':'Объем NLMK'}, inplace=True )

<ipython-input-72-5897c0291d41>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chmf.rename(columns = {'Date':'dt', 'Price':'Цена CHMF', 'Vol.':'Объем CHMF'}, inplace=True )
<ipython-input-72-5897c0291d41>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  magn.rename(columns = {'Дата':'dt', 'Цена':'Цена MAGN', 'Объём':'Объем MAGN'}, inplace=True )
<ipython-input-72-5897c0291d41>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nlmk.ren

In [73]:
price_d = {'K':1,
           'К':1,
           'M':1000,
           'М':1000}

def get_volume(s):
    try:
        s = '.'.join(s.split(','))
        s = '.'.join(s.split('.')[:2])
        return float(s[:-1]) * price_d[s[-1]]
    except AttributeError:
        return s

def det_price(s):
    try:
        s = ''.join(s.split(','))
        #s = '.'.join(s.split('.')[:2])
        return float(s)
    except AttributeError:
        return s

def get_date(s):
    try:
        s = '-'.join(s.split('/'))
        s = '-'.join(s.split('.'))
        try:
            return datetime.datetime.strptime(s, '%m-%d-%Y')
        except ValueError:
            return datetime.datetime.strptime(s, '%d-%m-%Y')
    except AttributeError:
        return s

In [74]:
stocks = [chmf, nlmk, magn]
volume_names = ['Объем CHMF', 'Объем NLMK', 'Объем MAGN']

for i in range(len(stocks)):
    stock = stocks[i]
    volume_name = volume_names[i]

    volumes = []
    for v in stock[volume_name]:
        volumes.append(get_volume(v))

    stock[volume_name] = volumes


price_stocks = [chmf, nlmk, magn, lme]
price_names = ['Цена CHMF', 'Цена NLMK', 'Цена MAGN', 'Цена LME']

for i in range(len(price_stocks)):
    stock = price_stocks[i]
    price_name = price_names[i]

    prices = []
    for p in stock[price_name]:
        prices.append(det_price(p))

    dates = []
    for d in stock['dt']:
        dates.append(get_date(d))

    stock[price_name] = prices
    stock['dt'] = dates
    stock = stock.dropna()
    stock = stock.sort_values(by='dt')

In [75]:
def from_timestamp(s):
    s = str(s).split(' ')[0]
    return int(s.split('-')[0]) * 365 + int(s.split('-')[1]) * 30 + int(s.split('-')[2])

In [76]:
dts = list(train.dt) + list(test.dt)

days_dts = []
for d in dts:
    days_dts.append(from_timestamp(d))

In [77]:
def check_nan(row, data):
    for key, value in row.items():
        if value == np.nan:
            value = data[key].mean()
        row[key] = value
    return row

In [78]:
def fill_nan(row):
    for key, _ in row.items():
        row[key] = np.nan
    return row

In [79]:
dfs = [macro, metal, price, mater, fuel, lme, magn, chmf, nlmk]

c = 0
for df in dfs:
    new_df = df.iloc[:1]

    if from_timestamp(df['dt'].iloc[0]) > days_dts[0]:
        print(from_timestamp(new_df['dt'].iloc[0]), days_dts[0])
        new_df.iloc[0] = fill_nan(new_df.iloc[0])

    new_df['dt'].iloc[0] = dts[0]

    ind = 0

    if c < 5:
        for i in range(1, len(dts)):
            if days_dts[i] < from_timestamp(df['dt'].iloc[ind]):
                new_df = new_df.append(check_nan(df.iloc[ind-1], new_df))

            else:
                while days_dts[i] > from_timestamp(df['dt'].iloc[ind]):
                    ind += 1
                new_df = new_df.append(check_nan(df.iloc[ind-1], new_df))

            if from_timestamp(new_df['dt'].iloc[-1]) > days_dts[i]:
                print('leak')
                break

            new_df['dt'].iloc[-1] = dts[i]

    else:
        for i in range(1, len(dts)):
            if days_dts[i] < from_timestamp(df['dt'].iloc[ind]):
                new_df = new_df.append(df.iloc[ind-1])

            else:
                while days_dts[i] > from_timestamp(df['dt'].iloc[ind]):
                    ind += 1
                new_df = new_df.append(df.iloc[ind-1])

            if from_timestamp(new_df['dt'].iloc[-1]) > days_dts[i]:
                print('leak')
                break

            new_df['dt'].iloc[-1] = dts[i]

    train = new_df.merge(train, on=['dt'])
    test = new_df.merge(test, on=['dt'])
    c += 1

735901 735510


Выходные данные были обрезаны до нескольких последних строк (5000).

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['dt'].iloc[-1] = dts[i]
<ipython-input-77-696e2b8e498e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row[key] = value
<ipython-input-79-220d335a8bb3>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(check_nan(df.iloc[ind-1], new_df))
<ipython-input-79-220d335a8bb3>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning

735961 735510


Выходные данные были обрезаны до нескольких последних строк (5000).
<ipython-input-79-220d335a8bb3>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['dt'].iloc[-1] = dts[i]
<ipython-input-77-696e2b8e498e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row[key] = value
<ipython-input-79-220d335a8bb3>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(check_nan(df.iloc[ind-1], new_df))
<ipython-input-79-220d335a8bb3>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

735544 735510


...

<ipython-input-79-220d335a8bb3>:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(df.iloc[ind-1])
<ipython-input-79-220d335a8bb3>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['dt'].iloc[-1] = dts[i]
<ipython-input-79-220d335a8bb3>:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(df.iloc[ind-1])
<ipython-input-79-220d335a8bb3>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['dt'

738516 735510


<ipython-input-79-220d335a8bb3>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['dt'].iloc[-1] = dts[i]
<ipython-input-79-220d335a8bb3>:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(df.iloc[ind-1])
<ipython-input-79-220d335a8bb3>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['dt'].iloc[-1] = dts[i]
<ipython-input-79-220d335a8bb3>:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.app

738516 735510


<ipython-input-79-220d335a8bb3>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['dt'].iloc[-1] = dts[i]
<ipython-input-79-220d335a8bb3>:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(df.iloc[ind-1])
<ipython-input-79-220d335a8bb3>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['dt'].iloc[-1] = dts[i]
<ipython-input-79-220d335a8bb3>:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.app

In [80]:
train

,dt,Цена NLMK,Объем NLMK,Цена CHMF,Объем CHMF,Цена MAGN,Объем MAGN,Цена LME,Цена на бензин,Цена на дт,...,Экспорт сырья Чугун_Всего отгружено с предприятия,Экспорт сырья Чугун_Литейный чугун,Экспорт сырья Чугун_Передельный чугун,Экспорт арматуры Сортовой прокат_Арматура,Экспорт арматуры Сортовой прокат_Катанка,Экспорт арматуры Сортовой прокат_Прутки прочие,Базовый индекс потребительских цен на товары и услуги,Ключевая ставка,Курс доллара,Цена на арматуру
0,2015-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32900
1,2015-01-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,97.0,68.0,50.0,NaN,NaN,NaN,32500
2,2015-01-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,97.0,68.0,50.0,NaN,NaN,NaN,32000
3,2015-01-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,97.0,68.0,50.0,NaN,NaN,NaN,30700
4,2015-02-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,97.0,68.0,50.0,NaN,NaN,NaN,30300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,2022-08-01,NaN,NaN,NaN,NaN,NaN,NaN,680.71,51.36,53.24,...,0.0,0.0,17.0,140.0,73.0,86.0,NaN,NaN,61.3101,47500
396,2022-08-08,NaN,NaN,NaN,NaN,NaN,NaN,641.00,51.35,53.21,...,0.0,8.0,57.0,147.0,87.0,52.0,NaN,NaN,60.3696,49000
397,2022-08-15,NaN,NaN,NaN,NaN,NaN,NaN,660.00,51.35,53.21,...,0.0,8.0,57.0,147.0,87.0,52.0,NaN,NaN,60.8993,47500
398,2022-08-22,NaN,NaN,NaN,NaN,NaN,NaN,645.00,51.35,53.21,...,0.0,8.0,57.0,147.0,87.0,52.0,NaN,NaN,59.1321,46500


In [81]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 400 entries, 0 to 399
Columns: 166 entries, dt to Цена на арматуру
dtypes: datetime64[ns](1), float64(164), int64(1)
memory usage: 521.9 KB


In [82]:
columns = ['dt']

for key, value in train.corr()['Цена на арматуру'].items():
    if (value > 0.4 or value < -0.4) and train[key].isnull().sum() < 40:
        print(value, " - ", key)
        columns.append(key)

<ipython-input-82-43450cbd7f1c>:3: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  for key, value in train.corr()['Цена на арматуру'].items():


0.6771029565548721  -  Концентрат коксующегося угля_Россия марка ОС FCA руб./т, без НДС
0.8920046646441515  -  Лом_HMS 1/2 80:20, FOB EC Роттердам, $/т
0.9239861070010841  -  Лом_3А, РФ CPT ж/д Южный ФО, руб./т, без НДС
0.9183928499556288  -  Лом_3А, РФ CPT ж/д Сибирский ФО, руб./т, без НДС
0.9310421437394831  -  Лом_3А, РФ CPT ж/д Центральный ФО, руб./т, без НДС
0.8265756737199672  -  ЖРС_Китай Iron ore fines Fe 62%, CFR
0.6894250098876626  -  Концентрат коксующегося угля_Россия марка КО FCA руб./т, без НДС
0.8982693051442053  -  ЖРС_Российские окатыши Fe 62-65,5%, SiO2 5,8-8,65, DAP Забайкальск-Манжули, $/т
0.8927375743642961  -  Лом_HMS 1/2 80:20, FOB США Восток, $/т
0.899083993578057  -  Лом_3А, FOB РФ Черное море, $/т
0.8143772149230237  -  Чугун_FOB Бразилия Север, $/т
0.9084129664518522  -  Лом_HMS 1/2 80:20, CFR ЮВА, $/т
0.6865382177203023  -  Концентрат коксующегося угля_Россия марка КС FCA руб./т, без НДС
0.925704169231091  -  Лом_3А, РФ FCA ж/д респ. Татарстан, руб./т, без Н

In [83]:
train_1 = train[columns]
test_1 = test[columns]

In [84]:
train_1 = train_1.dropna()

In [85]:
train_1.to_csv('/content/train_1_no_leaks_no_na.csv')
test_1.to_csv('/content/test_1_no_leaks_no_na.csv')

In [86]:
train_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 391 entries, 9 to 399
Data columns (total 73 columns):
 #   Column                                                                                            Non-Null Count  Dtype         
---  ------                                                                                            --------------  -----         
 0   dt                                                                                                391 non-null    datetime64[ns]
 1   Концентрат коксующегося угля_Россия марка ОС FCA руб./т, без НДС                                  391 non-null    float64       
 2   Лом_HMS 1/2 80:20, FOB EC Роттердам, $/т                                                          391 non-null    float64       
 3   Лом_3А, РФ CPT ж/д Южный ФО, руб./т, без НДС                                                      391 non-null    float64       
 4   Лом_3А, РФ CPT ж/д Сибирский ФО, руб./т, без НДС                                      

In [87]:
X, y = [], []
n = 3

for i in range(n, train_1.shape[0]-1):
    batch = []
    for j in range(n):
        batch += train_1.iloc[i-j].values.tolist()[1:]

    X.append(batch)
    y.append(train_1.iloc[i+1]['Цена на арматуру'])

X = np.asarray(X)
y = np.asarray(y)

In [25]:
file = open("/content/X", "wb")
np.save(file, X)
file.close

file = open("/content/y", "wb")
np.save(file, y)
file.close

<function BufferedWriter.close>

In [21]:
X.shape

(387, 216)

In [23]:
y.shape

(387,)

In [88]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [89]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [90]:
from catboost import CatBoostRegressor

# Initialize CatBoostRegressor
model = CatBoostRegressor(iterations=70,
                          learning_rate=0.1,
                          depth=12,
                          loss_function="RMSE",
                          #eval_metric='AUC'
                          )
# Fit model
model.fit(X_train, y_train, eval_set=(X_test, y_test))

0:	learn: 12366.9069709	test: 13790.4804838	best: 13790.4804838 (0)	total: 1.71s	remaining: 1m 58s
1:	learn: 11489.7288994	test: 12850.0010624	best: 12850.0010624 (1)	total: 2.76s	remaining: 1m 33s
2:	learn: 10762.5771824	test: 12079.7556756	best: 12079.7556756 (2)	total: 4.55s	remaining: 1m 41s
3:	learn: 10043.2398842	test: 11354.8683801	best: 11354.8683801 (3)	total: 6.36s	remaining: 1m 44s
4:	learn: 9346.7626280	test: 10610.8091107	best: 10610.8091107 (4)	total: 7.7s	remaining: 1m 40s
5:	learn: 8731.3843172	test: 9953.1068352	best: 9953.1068352 (5)	total: 8.98s	remaining: 1m 35s
6:	learn: 8181.5097138	test: 9360.8007272	best: 9360.8007272 (6)	total: 10.5s	remaining: 1m 34s
7:	learn: 7746.5618015	test: 8919.0131190	best: 8919.0131190 (7)	total: 11.9s	remaining: 1m 31s
8:	learn: 7275.8259502	test: 8451.1376031	best: 8451.1376031 (8)	total: 13.3s	remaining: 1m 29s
9:	learn: 6854.9679097	test: 8018.7574176	best: 8018.7574176 (9)	total: 15.1s	remaining: 1m 30s
10:	learn: 6478.6857012	tes

In [105]:
model.score(X_test, y_test)

0.9661900586300611

In [106]:
print(sum(((model.predict(X_test) - y_test) ** 2) ** 0.5) / y_test.shape[0])

1502.9797951144599


In [107]:
y_test

array([54500, 32900, 32500, 33500, 39300, 58000, 33300, 30700, 37500,
       23750, 28700, 29800, 38700, 21000, 77000, 31500, 71500, 28850,
       32900, 55000, 40300, 37500, 38200, 23800, 39300, 25400, 71900,
       34800, 38000, 26600, 45000, 32000, 24000, 35900, 38600, 59000,
       43500, 28000, 73500, 66000, 21300, 21800, 46500, 61000, 26550,
       26150, 74000, 31900, 23900, 24500, 40300, 33300, 33750, 33400,
       24500, 62000, 36000, 39500, 32800, 33500, 50500, 49000, 31700,
       48350, 23900, 21200, 26500, 65500, 33750, 62900, 32500, 26150,
       24700, 24700, 32500, 33000, 64000, 36800])

In [94]:
model.predict(X_test)

array([55390.35797275, 34086.19736587, 33736.35849248, 34179.81896895,
       39484.76835375, 49742.48583203, 32560.07329821, 31074.42780883,
       38043.02118898, 23531.06289786, 30279.85854118, 31742.97551565,
       38704.38454336, 22202.89551668, 71677.13103873, 33035.43115525,
       69610.20782688, 30654.91663784, 32234.27901079, 56630.58200152,
       27314.59305477, 38274.35283648, 37947.96981768, 24719.28697657,
       39531.08821325, 25141.31109275, 71236.63729283, 35446.10099714,
       38683.51638819, 25308.80110966, 43893.01882566, 31238.215927  ,
       24043.66054482, 35346.51103238, 37308.03547487, 58917.29618381,
       31683.76518705, 29610.33449224, 70960.76789497, 66933.84814834,
       22119.42071068, 22653.59559706, 45943.62438141, 56702.8713448 ,
       27933.47311839, 28362.89030184, 68569.24589876, 27375.49763069,
       24488.51759013, 23621.51174464, 39699.81940211, 34874.76005941,
       33768.99759741, 33041.46585363, 23618.61331837, 60867.24009349,
      

In [95]:
buy_volume = []

for i in range(train.shape[0]-10):
    actual_price = train['Цена на арматуру'].iloc[i]
    j = 1
    while actual_price < train['Цена на арматуру'].iloc[i+j] and j < 10:
        #print(actual_price, train['Цена на арматуру'].iloc[i+j])
        j += 1
    buy_volume.append(j)

volume_train = train.iloc[:-10]
volume_train['Объем закупки'] = buy_volume

<ipython-input-95-20ffcf04d88b>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  volume_train['Объем закупки'] = buy_volume


In [155]:
columns = ['dt']

for key, value in volume_train.corr()['Объем закупки'].items():
    if (value > 0.05 or value < -0.05) and volume_train[key].isnull().sum() < 10:
        print(value, " - ", key)
        columns.append(key)

0.0719420069737891  -  Лом_HMS 1/2 80:20, FOB EC Роттердам, $/т
0.05387217045480069  -  Лом_3А, РФ CPT ж/д Южный ФО, руб./т, без НДС
0.06792491504423277  -  Лом_3А, РФ CPT ж/д Сибирский ФО, руб./т, без НДС
0.05475676726432138  -  Лом_3А, РФ CPT ж/д Центральный ФО, руб./т, без НДС
0.0946585350608921  -  ЖРС_Китай Iron ore fines Fe 62%, CFR
0.07524880840747622  -  ЖРС_Российские окатыши Fe 62-65,5%, SiO2 5,8-8,65, DAP Забайкальск-Манжули, $/т
0.0722109202071474  -  Лом_HMS 1/2 80:20, FOB США Восток, $/т
0.06762660761319778  -  Лом_3А, FOB РФ Черное море, $/т
0.06261286614034334  -  Лом_HMS 1/2 80:20, CFR ЮВА, $/т
0.05554724429763603  -  Лом_3А, РФ FCA ж/д респ. Татарстан, руб./т, без НДС
0.0902016109776067  -  ЖРС_Российский концентрат Fe 62,5-67,5%, SiO2 0,78-8,51, сухой вес, DAP Забайкальск-Манжули, $/т
0.05481133927429454  -  Лом_3А, РФ FCA ж/д Московский регион, руб./т, без НДС
0.07160179118839777  -  Чугун_FOB Россия Черное море, $/т
0.060799097676455915  -  Лом_3А, РФ CPT ж/д Ураль

<ipython-input-155-7ac633842236>:3: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  for key, value in volume_train.corr()['Объем закупки'].items():


In [97]:
volume_train_cutted = volume_train[columns]

In [98]:
volume_train_cutted.corr()

<ipython-input-98-6ad921f796f7>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  volume_train_cutted.corr()


,Производство сырья ЖРС_Аглоруда,Производство сырья Концентрат коксующегося угля_Коксовые,Производство сырья Металлизованная продукция_ГБЖ,Производство сырья Металлизованная продукция_ПВЖ,Производство сырья Рядовой коксующийся уголь_Жирные,Производство сырья Сталь_Конвертер,Производство сырья Сталь_Мартен,ПотреблениеЖРС_Концентрат,ПотреблениеЖРС_Концентрат для дообогащения,ПотреблениеЖРС_Аглоруда,...,Спрос Металлизованная продукция_ГБЖ,Спрос Концентрат коксующегося угля_Коксовые,Импорт сырья Концентрат коксующегося угля_Коксовые,Экспорт сырья ЖРС_Аглоруда,Экспорт сырья ЖРС_Концентрат,Экспорт сырья Концентрат коксующегося угля_Отощающие,Экспорт сырья Металлизованная продукция_ГБЖ,Экспорт сырья Чугун_Передельный чугун,Экспорт арматуры Сортовой прокат_Арматура,Объем закупки
Производство сырья ЖРС_Аглоруда,1.000000,0.007305,0.357091,0.018136,0.443638,0.341921,0.289297,-0.103092,-0.170291,0.843082,...,0.204154,-0.105713,0.420043,-0.046778,0.416919,0.091015,0.444968,0.164625,0.344838,0.172409
Производство сырья Концентрат коксующегося угля_Коксовые,0.007305,1.000000,0.074350,-0.039751,-0.080332,0.216835,0.353322,0.274940,-0.030864,0.144842,...,0.192929,0.242439,0.399237,0.054686,0.129450,-0.142451,-0.077846,0.014365,0.085335,-0.119979
Производство сырья Металлизованная продукция_ГБЖ,0.357091,0.074350,1.000000,0.266278,0.447696,0.384553,0.407102,0.375202,-0.662194,0.473403,...,0.740109,-0.517077,0.181440,0.386244,0.651411,0.100399,0.357196,0.286284,0.394276,0.159303
Производство сырья Металлизованная продукция_ПВЖ,0.018136,-0.039751,0.266278,1.000000,0.366872,0.311284,-0.016306,0.051891,-0.097384,0.019640,...,0.112375,-0.246683,-0.034564,0.029292,0.131908,0.052609,0.189774,0.107899,0.196294,0.131142
Производство сырья Рядовой коксующийся уголь_Жирные,0.443638,-0.080332,0.447696,0.366872,1.000000,0.471686,-0.015049,0.099611,-0.215399,0.348107,...,0.070715,-0.413601,0.164523,0.013742,0.535748,0.259259,0.721854,0.409305,0.470337,0.111586
Производство сырья Сталь_Конвертер,0.341921,0.216835,0.384553,0.311284,0.471686,1.000000,0.084229,0.359457,-0.149984,0.394698,...,0.202440,-0.086416,0.322420,-0.064706,0.367418,0.118544,0.514446,0.339557,0.520970,0.151384
Производство сырья Сталь_Мартен,0.289297,0.353322,0.407102,-0.016306,-0.015049,0.084229,1.000000,0.009987,-0.421933,0.347830,...,0.504353,-0.126239,0.320382,0.162377,0.312481,0.022688,0.012822,0.079385,0.062906,0.105970
ПотреблениеЖРС_Концентрат,-0.103092,0.274940,0.375202,0.051891,0.099611,0.359457,0.009987,1.000000,-0.173739,0.050802,...,0.149945,0.053682,-0.106800,0.304257,0.195908,-0.208584,0.024835,0.133384,0.146674,-0.107557
ПотреблениеЖРС_Концентрат для дообогащения,-0.170291,-0.030864,-0.662194,-0.097384,-0.215399,-0.149984,-0.421933,-0.173739,1.000000,-0.253882,...,-0.514615,0.470412,-0.039404,-0.417627,-0.553864,-0.160515,-0.178578,-0.169058,-0.156940,-0.113553
ПотреблениеЖРС_Аглоруда,0.843082,0.144842,0.473403,0.019640,0.348107,0.394698,0.347830,0.050802,-0.253882,1.000000,...,0.282026,-0.162547,0.478309,0.063075,0.408167,0.021147,0.408765,0.188879,0.312627,0.111204


In [99]:
volume_train_cutted

,dt,Производство сырья ЖРС_Аглоруда,Производство сырья Концентрат коксующегося угля_Коксовые,Производство сырья Металлизованная продукция_ГБЖ,Производство сырья Металлизованная продукция_ПВЖ,Производство сырья Рядовой коксующийся уголь_Жирные,Производство сырья Сталь_Конвертер,Производство сырья Сталь_Мартен,ПотреблениеЖРС_Концентрат,ПотреблениеЖРС_Концентрат для дообогащения,...,Спрос Металлизованная продукция_ГБЖ,Спрос Концентрат коксующегося угля_Коксовые,Импорт сырья Концентрат коксующегося угля_Коксовые,Экспорт сырья ЖРС_Аглоруда,Экспорт сырья ЖРС_Концентрат,Экспорт сырья Концентрат коксующегося угля_Отощающие,Экспорт сырья Металлизованная продукция_ГБЖ,Экспорт сырья Чугун_Передельный чугун,Экспорт арматуры Сортовой прокат_Арматура,Объем закупки
0,2015-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,2015-01-12,376.0,2000.0,243.0,266.0,3886.0,3803.0,102.0,3666.0,593.0,...,39.0,1454.0,16.0,0.0,0.0,0.0,0.0,0.0,97.0,1
2,2015-01-19,376.0,2000.0,243.0,266.0,3886.0,3803.0,102.0,3666.0,593.0,...,39.0,1454.0,16.0,0.0,0.0,0.0,0.0,0.0,97.0,1
3,2015-01-26,376.0,2000.0,243.0,266.0,3886.0,3803.0,102.0,3666.0,593.0,...,39.0,1454.0,16.0,0.0,0.0,0.0,0.0,0.0,97.0,1
4,2015-02-02,376.0,2000.0,243.0,266.0,3886.0,3803.0,102.0,3666.0,593.0,...,39.0,1454.0,16.0,0.0,0.0,0.0,0.0,0.0,97.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,2022-05-23,456.0,2061.0,358.0,267.0,3850.0,3841.0,140.0,3453.0,479.0,...,59.0,1221.0,38.0,8.0,361.0,0.0,18.0,46.0,166.0,1
386,2022-05-30,456.0,2061.0,358.0,267.0,3850.0,3841.0,140.0,3453.0,479.0,...,59.0,1221.0,38.0,8.0,361.0,0.0,18.0,46.0,166.0,1
387,2022-06-06,529.0,2363.0,427.0,297.0,4772.0,4127.0,138.0,4002.0,550.0,...,72.0,1639.0,24.0,12.0,527.0,0.0,7.0,41.0,213.0,1
388,2022-06-13,529.0,2363.0,427.0,297.0,4772.0,4127.0,138.0,4002.0,550.0,...,72.0,1639.0,24.0,12.0,527.0,0.0,7.0,41.0,213.0,10


In [137]:
X_v, y_v = [], []
n = 3

for i in range(n, volume_train_cutted.shape[0]):
    batch = []
    for j in range(n):
        batch += volume_train_cutted.iloc[i-j].values.tolist()[1:-1]

    X_v.append(batch)
    y_v.append(volume_train_cutted.iloc[i]['Объем закупки'])

X_v = np.asarray(X_v)
y_v = np.asarray(y_v)

In [139]:
X_v.shape

(387, 81)

In [140]:
Xv_train, Xv_test, yv_train, yv_test = train_test_split(X_v, y_v, test_size=0.2, random_state=42)

In [141]:
from catboost import CatBoostClassifier

model_vc = CatBoostClassifier(iterations=100,
                           learning_rate=0.1,
                           depth=12,
                           loss_function='MultiClass')

model_vc.fit(Xv_train, yv_train, eval_set=(Xv_test, yv_test))

0:	learn: 2.0979599	test: 2.0819937	best: 2.0819937 (0)	total: 56.6ms	remaining: 5.61s
1:	learn: 2.0219805	test: 2.0186120	best: 2.0186120 (1)	total: 2.01s	remaining: 1m 38s
2:	learn: 1.9541302	test: 1.9620126	best: 1.9620126 (2)	total: 3.92s	remaining: 2m 6s
3:	learn: 1.8938868	test: 1.9162392	best: 1.9162392 (3)	total: 5.85s	remaining: 2m 20s
4:	learn: 1.8385442	test: 1.8775938	best: 1.8775938 (4)	total: 8.93s	remaining: 2m 49s
5:	learn: 1.7773063	test: 1.8267639	best: 1.8267639 (5)	total: 11.6s	remaining: 3m 1s
6:	learn: 1.7215634	test: 1.7870328	best: 1.7870328 (6)	total: 13.5s	remaining: 2m 59s
7:	learn: 1.6733940	test: 1.7495709	best: 1.7495709 (7)	total: 15.4s	remaining: 2m 57s
8:	learn: 1.6243691	test: 1.7140431	best: 1.7140431 (8)	total: 17.3s	remaining: 2m 55s
9:	learn: 1.5802386	test: 1.6808852	best: 1.6808852 (9)	total: 19.3s	remaining: 2m 53s
10:	learn: 1.5398874	test: 1.6524085	best: 1.6524085 (10)	total: 21.6s	remaining: 2m 54s
11:	learn: 1.4997519	test: 1.6295722	best: 

In [142]:
model_vc.score(Xv_test, yv_test)

0.6794871794871795

In [143]:
model_vc.predict(Xv_test)

array([[ 1],
       [10],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [10],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [10],
       [ 1],
       [ 1],
       [ 1],
       [ 3],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [10],
       [ 1],
       [10],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [10],
       [ 1],
       [ 1],
       [ 1],
       [10],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [10],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [10],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 5],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [10],

In [144]:
yv_test

array([ 1,  5,  1, 10,  1,  1,  1,  1, 10,  1,  1, 10,  4,  1,  4, 10, 10,
        1,  1,  1, 10,  1,  1,  1, 10,  2, 10,  1,  1,  1,  1,  1,  1, 10,
       10,  1, 10,  1, 10,  8,  3,  1, 10,  1,  1,  1,  1, 10,  4,  1,  1,
        1,  1,  1,  1,  1,  1, 10,  1,  4,  1,  1,  1, 10,  1,  1,  1,  1,
       10,  1,  1,  1,  1, 10,  1,  1,  1, 10])

In [145]:
Xv_test.shape

(78, 81)

In [146]:
test["Объем закупки"] = 0
testv = test[columns]
testv.drop(["Объем закупки"], axis=1, inplace=True)

<ipython-input-146-36e6d080e39f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testv.drop(["Объем закупки"], axis=1, inplace=True)


In [148]:
n = 3
X_testv = []

for i in range(testv.shape[0]):
    batch = []
    for j in range(n):
        batch += testv.iloc[max(0, i-j)].values.tolist()[1:]

    print(model_vc.predict(np.asarray(batch)))
    X_testv.append(batch)

X_testv = np.asarray(X_testv)

[10]
[10]
[10]
[10]
[10]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]


In [149]:
model_vc.predict(X_testv)

array([[10],
       [10],
       [10],
       [10],
       [10],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1]])

In [156]:
test["Цена на арматуру"]

0     46000
1     46000
2     46000
3     45500
4     45500
5     45000
6     44000
7     43000
8     42000
9     41500
10    41000
11    40000
12    40000
13    40500
14    41700
15    41700
16    42500
17    42700
18    42700
19    42700
20    42800
21    42800
22    44500
23    44500
24    46000
25    49200
26    53700
27    55500
Name: Цена на арматуру, dtype: int64